# This File to load daily data (1 min) into MongoDB

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime

In [2]:
def downloader_1(stock):
    data=getdata(stock,'Market1')
    start=data.tail(1).index[0]
    # timedelta=datetime.timedelta(days=4)
    # start=start+timedelta
    exchange=['.NS','.BO']
    try:
       # df_ns=yf.download(stock+exchange[0]).reset_index()
        ns=yf.download(stock+exchange[0],start=start,interval='1m')
        df_ns = ns.iloc[1: , :].copy()
        del ns    
    except:
        print(f'{stock+exchange[0]} not found')
            
    try:
        bo=yf.download(stock+exchange[1],start=start,interval='1m')
        df_bo = bo.iloc[1: , :].copy()
        del bo
        #df_bo=yf.download(stock+exchange[1]).reset_index()
    except:
        print(f'{stock+exchange[1]} not found')
    if not df_ns.empty:
        # print ('df_ns not empty')
        if not  df_bo.empty:
            # df_bo['Date']=pd.to_datetime(df_bo['Date'])
#             df_bo=df_bo.set_index('Date')
#             df_ns['Date']=pd.to_datetime(df_ns['Date'])
#             df_ns=df_ns.set_index('Date')
            # print ('df_bo not empty')
            if df_bo.shape[0]>1 & df_ns.shape[0]>1:
                # print ('ne and bo both having data')
            
                df_ns.dropna(axis=0,how='any',inplace=True)
                df_bo.dropna(axis=0,how='any',inplace=True)
                df_ns['Volume']=df_ns['Volume'].astype(int)
                df_bo['Volume']=df_bo['Volume'].astype(int)
                df=pd.merge(df_ns,df_bo,on='Datetime',how='left')
                df=df.fillna(0)
                df['Volume']=df['Volume_x']+df['Volume_y']
                df=df[['Open_x','High_x','Low_x','Adj Close_x','Volume']]
                df=df.reset_index()
                df=df.rename(columns={'Open_x':'Open','High_x':'High','Low_x':'Low','Adj Close_x':'Close'})
                
                if df.iloc[-1: , -1:].values[0,0]==0:
                    df.drop(df.tail(1).index,inplace=True)
                return df
            elif df_ns.shape[0]>1:
                df_ns.dropna(axis=0,how='any',inplace=True)
                df_ns['Volume']=df_ns['Volume'].astype(int)
                df_ns=df_ns[['Open','High','Low','Adj Close','Volume']]
                df_ns=df_ns.reset_index()
                df_ns=df_ns.rename(columns={'Adj Close':'Close'})

                if df_ns.iloc[-1: , -1:].values[0,0]==0:
                    df_ns.drop(df_ns.tail(1).index,inplace=True)
                return df_ns
            else:
                df=pd.DataFrame()
                return df
                
        else:
            if df_ns.shape[0]>1:
                df_ns.dropna(axis=0,how='any',inplace=True)
                df_ns['Volume']=df_ns['Volume'].astype(int)
                df_ns=df_ns[['Open','High','Low','Adj Close','Volume']]
                df_ns=df_ns.reset_index()
                df_ns=df_ns.rename(columns={'Adj Close':'Close'})

                if df_ns.iloc[-1: , -1:].values[0,0]==0:
                    df_ns.drop(df_ns.tail(1).index,inplace=True)
                return df_ns
        # elif df_ns.shape[0]>1:
        #     df_ns.dropna(axis=0,how='any',inplace=True)
        #     df_ns['Volume']=df_ns['Volume'].astype(int)
        #     df_ns=df_ns[['Open','High','Low','Adj Close','Volume']]
        #     df_ns=df_ns.reset_index()
        #     df_ns=df_ns.rename(columns={'Adj Close':'Close'})
        #     if df_ns.iloc[-1: , -1:].values[0,0]==0:
        #         df_ns.drop(df_ns.tail(1).index,inplace=True)
        #     return df_ns
    else:
        df=pd.DataFrame()
        return df

In [3]:
#This method read data from Mongo DB
def getdata(stock,db):
    stock=str(stock)
    mongo_client=MongoClient()
    db=mongo_client[db]
    data=db[stock].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    ###Covert to UTC to get the correct Values
    df['Datetime']=pd.to_datetime(df['Datetime']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
    #df['Datetime']=pd.to_datetime(df['Datetime'])
    df=df.set_index('Datetime')
    return df

In [4]:
# This Method load data into DB.
def to_db_1(stocklist):
    m_client=MongoClient()
    db=m_client["Market1"]
    print(f'1 min data Loading Starting')
    for stock in stocklist:
        print(f'{stock} 1 min data loading')
        df=downloader_1(stock)
        try:
            if not df.empty:
                data=df.to_dict(orient='records')
                db[stock].insert_many(data)
                print(f'{stock} loading completed')
            else:
                print(f'{stock} Nothing to load')
        except:
            print(f'{stock} load Failed')
            
    print(f'1 min data Loading Ended')
    m_client.close()
    

In [5]:
# To Fetch Stocks from DB
def get_stocklist(tbl):
    mongo_client=MongoClient()
    db=mongo_client["NIFTY500"]
    data=db[tbl].find()
    df=pd.DataFrame(list(data))
    df.drop("_id",axis=1,inplace=True)
    return df

In [6]:
df=get_stocklist('NIFTY500')
stocks=df.Symbol.to_list()

In [7]:
#provide list od stocks you want to downlad and load into mongo db
#stocks=['AAVAS']
#Calling load to db method 

to_db_1(stocks)



1 min data Loading Starting
3MINDIA 1 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
3MINDIA loading completed
ABB 1 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ABB loading completed
POWERINDIA 1 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
POWERINDIA loading completed
ACC 1 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ACC loading completed
AIAENG 1 min data loading
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
AIAENG loading completed
APLAPOLLO 1 min data loading
